<a href="https://colab.research.google.com/github/bswaika/kaggle_hm_recommender_competition/blob/master/tc_recommender_04_01_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
kaggle_file = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!rm -rf ./sample_data && mkdir -p ~/.kaggle && cp ./kaggle.json ~/.kaggle/kaggle.json && rm ./kaggle.json && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!ls -ah ~ && ls -ah .

.   .bashrc  .config  .ipython	.kaggle  .local  .profile
..  .cache   .gsutil  .jupyter	.keras	 .npm	 .tmux.conf
.  ..  .config


In [ ]:
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations

100% 28.7G/28.7G [02:11<00:00, 214MB/s]
100% 28.7G/28.7G [02:11<00:00, 235MB/s]


In [ ]:
!zip -d h-and-m-personalized-fashion-recommendations.zip images/*/*.jpg && unzip h-and-m-personalized-fashion-recommendations.zip && rm h-and-m-personalized-fashion-recommendations.zip

Streaming output truncated to the last 5000 lines.
deleting: images/076/0763743002.jpg
deleting: images/076/0763743003.jpg
deleting: images/076/0763754001.jpg
deleting: images/076/0763756001.jpg
deleting: images/076/0763763001.jpg
deleting: images/076/0763777001.jpg
deleting: images/076/0763779001.jpg
deleting: images/076/0763785001.jpg
deleting: images/076/0763785002.jpg
deleting: images/076/0763785003.jpg
deleting: images/076/0763789001.jpg
deleting: images/076/0763789002.jpg
deleting: images/076/0763789003.jpg
deleting: images/076/0763802001.jpg
deleting: images/076/0763806001.jpg
deleting: images/076/0763807001.jpg
deleting: images/076/0763807002.jpg
deleting: images/076/0763807003.jpg
deleting: images/076/0763811006.jpg
deleting: images/076/0763821001.jpg
deleting: images/076/0763833003.jpg
deleting: images/076/0763836001.jpg
deleting: images/076/0763836003.jpg
deleting: images/076/0763842002.jpg
deleting: images/076/0763842005.jpg
deleting: images/076/0763842007.jpg
deleting: ima

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




deleting: images/091/0916861001.jpg
deleting: images/091/0916861002.jpg
deleting: images/091/0916866001.jpg
deleting: images/091/0916866002.jpg
deleting: images/091/0916866003.jpg
deleting: images/091/0916881001.jpg
deleting: images/091/0916881002.jpg
deleting: images/091/0916916001.jpg
deleting: images/091/0916917001.jpg
deleting: images/091/0916917002.jpg
deleting: images/091/0916926001.jpg
deleting: images/091/0916926002.jpg
deleting: images/091/0916926003.jpg
deleting: images/091/0916926005.jpg
deleting: images/091/0916926006.jpg
deleting: images/091/0916926009.jpg
deleting: images/091/0916943001.jpg
deleting: images/091/0916993002.jpg
deleting: images/091/0916995002.jpg
deleting: images/091/0916995003.jpg
deleting: images/091/0917000001.jpg
deleting: images/091/0917000002.jpg
deleting: images/091/0917000003.jpg
deleting: images/091/0917016001.jpg
deleting: images/091/0917053002.jpg
deleting: images/091/0917056001.jpg
deleting: images/091/0917084001.jpg
deleting: images/091/091712

In [ ]:
!pip install turicreate

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libnvrtc8.0
E: Couldn't find any package by glob 'libnvrtc8.0'
E: Couldn't find any package by regex 'libnvrtc8.0'
     |████████████████████████████████| 274.9 MB 4.3 kB/s 
     |████████████████████████████████| 88 kB 9.0 MB/s 
     |████████████████████████████████| 5.0 MB 39.8 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
     |████████████████████████████████| 132 kB 64.1 MB/s 
  ERROR: Failed building wheel for numpy
  Running setup.py clean for numpy
  ERROR: Failed cleaning build dir for numpy
Failed to build numpy
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: idna
    Found existing installation: idna 2.10
    Uninstalling idna-2.10:
      Successfully uninstalled idna-2.10
  Attempting uninstall: request

In [ ]:
import pandas as pd
import turicreate as tc

tc.config.set_num_gpus(1)

In [ ]:
class DataParser:
  def __init__(self, filepaths):
    assert isinstance(filepaths, dict)
    assert 'articles' in filepaths
    assert 'customers' in filepaths
    assert 'transactions' in filepaths
    self._filepaths = filepaths
    self._raw = {}
    # self._associations = {}
    self._data = {}
    self._encoders = {
      'articles': self.encode_articles,
      'customers': self.encode_customers,
      'transactions': self.encode_transactions,
    }
  
  def load_data(self, index):
    assert index in ['articles', 'customers', 'transactions']
    self._raw[index] = pd.read_csv(self._filepaths[index])
  
  def encode_data(self, index):
    assert index in ['articles', 'customers', 'transactions']
    assert index in self._raw
    self._encoders[index]()

  def create_association(self, index, id, name):  
    association_dict = self._raw[index][[id, name]]
    association_dict = association_dict.set_index([id]).to_dict()[name]
    return pd.DataFrame.from_dict(association_dict, orient='index', columns=['name'])
  
  def encode_articles(self):
    selectors = [col for col in self._raw['articles'].columns if any(['id' in col, 'code' in col, 'no' in col])]
    self._data['articles'] = self._raw['articles'].loc[:, selectors].copy(deep=True)

    # self._associations['product_types'] = self.create_association('articles', 'product_type_no', 'product_type_name')
    # self._associations['departments'] = self.create_association('articles', 'department_no', 'department_name')
    # self._associations['index_groups'] = self.create_association('articles', 'index_group_no', 'index_group_name')
    # self._associations['sections'] = self.create_association('articles', 'section_no', 'section_name')
    # self._associations['garment_groups'] = self.create_association('articles', 'garment_group_no', 'garment_group_name')
    # self._associations['graphical_appearances'] = self.create_association('articles', 'graphical_appearance_no', 'graphical_appearance_name')
    # self._associations['colour_groups'] = self.create_association('articles', 'colour_group_code', 'colour_group_name')
    # self._associations['indexes'] = self.create_association('articles', 'index_code', 'index_name')
    # self._associations['perceived_colour_values'] = self.create_association('articles', 'perceived_colour_value_id', 'perceived_colour_value_name')
    # self._associations['perceived_colour_masters'] = self.create_association('articles', 'perceived_colour_master_id', 'perceived_colour_master_name')

    self._data['articles']['index_code'] = self._data['articles']['index_code'].astype('category')
    # self._associations['index_codes'] = self._data['articles']['index_code'].cat.categories
    self._data['articles']['index_code'] = self._data['articles']['index_code'].cat.codes
  
  def fill_customers(self):
    self._raw['customers']['age'].fillna(0.0, inplace=True)
    self._raw['customers']['club_member_status'].fillna('NONE', inplace=True)
    self._raw['customers']['Active'].fillna(0.0, inplace=True)
    self._raw['customers']['FN'].fillna(0.0, inplace=True)
    self._raw['customers']['fashion_news_frequency'].fillna('NONE', inplace=True)
    self._raw['customers']['fashion_news_frequency'] = self._raw['customers']['fashion_news_frequency'].str.replace('NONE', 'None')
  
  def encode_customers(self):
    self.fill_customers()
    self._data['customers'] = self._raw['customers'].copy(deep=True)

    self._data['customers']['fashion_news_frequency'] = self._data['customers']['fashion_news_frequency'].astype('category')
    self._data['customers']['club_member_status'] = self._data['customers']['club_member_status'].astype('category')
    self._data['customers']['Active'] = self._data['customers']['Active'].astype('category')
    self._data['customers']['FN'] = self._data['customers']['FN'].astype('category')

    # self._associations['member_statuses'] = self._data['customers']['club_member_status'].cat.categories
    self._data['customers']['club_member_status'] = self._data['customers']['club_member_status'].cat.codes

    # self._associations['news_freqs'] = self._data['customers']['fashion_news_frequency'].cat.categories
    self._data['customers']['fashion_news_frequency'] = self._data['customers']['fashion_news_frequency'].cat.codes
  
  def encode_transactions(self):
    self._data['transactions'] = self._raw['transactions'].copy(deep=True)

  def get_data(self, index) -> pd.DataFrame:
    assert index in ['articles', 'customers', 'transactions']
    if index not in self._raw and index not in self._data:
      self.load_data(index)
      print(f'Loaded {index}')
    if index not in self._data:
      self.encode_data(index)
      del self._raw[index]
    return self._data[index]
  
  # def get_association(self, index) -> pd.DataFrame:
  #   assert index in self._associations
  #   return self._associations[index]

In [ ]:
filepaths = {
    'articles': './articles.csv',
    'customers': './customers.csv',
    'transactions': './transactions_train.csv'
}

In [ ]:
parser = DataParser(filepaths)

In [ ]:
articles = parser.get_data('articles')
customers = parser.get_data('customers')
transactions = parser.get_data('transactions')

Loaded articles
Loaded customers
Loaded transactions


In [ ]:
transactions = transactions.loc[int(0.3*transactions.shape[0]):, :].reset_index()[['t_dat', 'customer_id', 'article_id']]
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'], format='%Y-%m-%d')

In [ ]:
transactions.shape

(22251827, 3)

In [ ]:
train = transactions.groupby(['customer_id', 'article_id'])[['t_dat']].count().reset_index().rename(columns={'t_dat': 'purchase_count'})
train = train.merge(train.groupby(['customer_id'])['purchase_count'].sum().reset_index().rename(columns={'purchase_count': 'total'}), how='outer', on='customer_id')
train['weights'] = train['purchase_count'] / train['total']
del transactions
del parser

In [ ]:
articles = tc.SFrame(data=articles.to_dict(orient='list'))
customers = tc.SFrame(data=customers.to_dict(orient='list'))

In [ ]:
train = train.to_dict(orient='list')

In [ ]:
train = tc.SFrame(data=train)

In [ ]:
articles.head()

article_id,colour_group_code,department_no,garment_group_no,graphical_appearance_no,index_code,index_group_no
108775015,9,1676,1002,1010016,0,1
108775044,10,1676,1002,1010016,0,1
108775051,11,1676,1002,1010017,0,1
110065001,9,1339,1017,1010016,1,1
110065002,10,1339,1017,1010016,1,1
110065011,12,1339,1017,1010016,1,1
111565001,9,3608,1021,1010016,1,1
111565003,13,3608,1021,1010016,1,1
111586001,9,3608,1021,1010016,1,1
111593001,9,3608,1021,1010016,1,1


In [ ]:
customers.head()

Active,FN,age,club_member_status,customer_id,fashion_news_frequency
0.0,0.0,49.0,0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3 ...,1
0.0,0.0,25.0,0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e ...,1
0.0,0.0,24.0,0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca ...,1
0.0,0.0,54.0,0,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2 ...,1
1.0,1.0,52.0,0,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f ...,2
0.0,0.0,0.0,2,000064249685c11552da43ef22a5030f35a147f723d5b0 ...,1
0.0,0.0,20.0,0,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d ...,1
1.0,1.0,32.0,0,00007d2de826758b65a93dd24ce629ed66842531df6699 ...,2
0.0,0.0,20.0,0,00007e8d4e54114b5b2a9b51586325a8d0fa74ea23ef77 ...,1
0.0,0.0,20.0,0,00008469a21b50b3d147c97135e25b4201a8c58997f787 ...,1


In [ ]:
model = tc.recommender.item_similarity_recommender.create(train[['customer_id', 'article_id', 'weights']], user_id='customer_id', item_id='article_id', target='weights', user_data=customers, item_data=articles, only_top_k=12, similarity_type='cosine', target_memory_usage=1589934592)

Preparing data set.

Data has 19175371 observations with 1371980 users and 105542 items.

Data prepared in: 22.4948s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 2m 5s                               | 100              | 105542          |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 129.411s

In [ ]:
popular_items = train.groupby('article_id', operations={'total': tc.aggregate.SUM('total')})
popular_items.head(10)

article_id,total
591334029,12891
479604005,1018
744239005,1383
215303002,7239
789689004,43892
658880003,134
783346006,136645
860797001,44882
783732003,4138
926306002,26515


In [ ]:
popular_items = popular_items.sort('total', ascending=False)

In [ ]:
popular_items = popular_items.to_dataframe()

In [ ]:
predictions = model.recommend(customers['customer_id'], diversity=1, random_seed=9196).to_dataframe()

recommendations finished on 1000/1371980 queries. users per second: 666.866

recommendations finished on 2000/1371980 queries. users per second: 722.171

recommendations finished on 3000/1371980 queries. users per second: 747.302

recommendations finished on 4000/1371980 queries. users per second: 758.209

recommendations finished on 5000/1371980 queries. users per second: 764.707

...

recommendations finished on 1367000/1371980 queries. users per second: 805.404

recommendations finished on 1368000/1371980 queries. users per second: 805.461

recommendations finished on 1369000/1371980 queries. users per second: 805.524

recommendations finished on 1370000/1371980 queries. users per second: 805.585

recommendations finished on 1371000/1371980 queries. users per second: 805.642

In [ ]:
predictions.head()

,customer_id,article_id,score,rank
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,871878001,0.001353,1
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,694736008,0.001244,2
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,871869001,0.001188,3
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,811290003,0.000902,4
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,577588002,0.000868,5


In [ ]:
predictions.to_csv('./test.csv')

In [ ]:
popular_items.head()

,article_id,total
0,706016001,1228115
1,759871002,953342
2,720125001,884563
3,706016002,844913
4,610776002,808007


In [ ]:
popular_items.to_csv('./popular_items.csv')

In [ ]:
!ls

articles.csv   drive		  sample_submission.csv  transactions_train.csv
customers.csv  popular_items.csv  test.csv


In [ ]:
from google.colab import files

files.download('popular_items.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>